In [ ]:
import numpy as np

In [ ]:
P = np.array([[0.7, 0.3, 0, 0, 0, 0],
              [0.2, 0.6, 0.2, 0, 0, 0],
              [0, 0, 0.9, 0.1, 0, 0],
              [0, 0, 0, 0.7, 0.3, 0],
              [0, 0, 0, 0, 0.8, 0.2],
              [0.4, 0, 0, 0, 0, 0.6]])